In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import functions as f

In [3]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions",spark.sparkContext.defaultParallelism*2)
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'LEGACY')
spark.conf.set('spark.sql.legacy.parquet.int96RebaseModeInWrite', 'LEGACY')

In [5]:
%%sh
ls /mnt/hgfs/dri/dwSpark/parquet/

DW_DRI.DWD_CALENDARIO.parquet
DW_DRI.DWD_COTACAO.parquet
DW_DRI.DWD_EDITAL.parquet
DW_DRI.DWD_ELEMENTO_DESPESA.parquet
DW_DRI.DWD_GEOGRAFIA.parquet
DW_DRI.DWD_INSTITUICAO_ENSINO.parquet
DW_DRI.DWD_MODALIDADE_BOLSA.parquet
DW_DRI.DWD_MOEDA.parquet
DW_DRI.DWD_PESSOA.parquet
DW_DRI.DWD_PROCESSO_AUXPE.parquet
DW_DRI.DWD_PROCESSO_BOLSA.parquet
DW_DRI.DWD_PROCESSO_PROJETO.parquet
DW_DRI.DWD_PROGRAMA.parquet
DW_DRI.DWD_RUBRICA.parquet
DW_DRI.DWD_SITUACAO_PRESTACAO.parquet
DW_DRI.DWD_SITUACAO_PROCESSO.parquet
DW_DRI.DWD_TIPO_BENEFICIO.parquet
DW_DRI.DWD_TIPO_SOLICITACAO.parquet
DW_DRI.DWD_UNIDADE_ORGANIZACIONAL.parquet
DW_DRI.DWF_EXTRATO_CARTAO_PESQUISADOR.parquet
DW_DRI.DWF_PAGAMENTO.parquet
DW_DRI.DWF_PRESTACAO_CONTA.parquet


In [64]:
dwd_calendario = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_CALENDARIO.parquet')
dwd_edital = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_EDITAL.parquet')
dwd_elemento_despesa = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_ELEMENTO_DESPESA.parquet')
dwd_geografia = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_GEOGRAFIA.parquet')
dwd_instituicao_ensino = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_INSTITUICAO_ENSINO.parquet')
dwd_modalidade_bolsa = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_MODALIDADE_BOLSA.parquet')
dwd_moeda = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_MOEDA.parquet')
dwd_pessoa = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_PESSOA.parquet')
dwd_processo_projeto = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_PROCESSO_PROJETO.parquet')
dwd_processo_bolsa = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_PROCESSO_BOLSA.parquet')
dwd_processo_auxpe = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_PROCESSO_AUXPE.parquet')
dwd_programa = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_PROGRAMA.parquet')
dwd_rubrica = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_RUBRICA.parquet')
dwd_situacao_prestacao = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_SITUACAO_PRESTACAO.parquet')
dwd_situacao_processo = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_SITUACAO_PROCESSO.parquet')
dwd_tipo_beneficio = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_TIPO_BENEFICIO.parquet')
dwd_tipo_solicitacao = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_TIPO_SOLICITACAO.parquet')
dwd_unidade_organizacional = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWD_UNIDADE_ORGANIZACIONAL.parquet')
dwf_extrato_cartao_pesquisador = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWF_EXTRATO_CARTAO_PESQUISADOR.parquet')
dwf_pagamento = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWF_PAGAMENTO.parquet')
dwf_prestacao_conta = spark.read.parquet('/mnt/hgfs/dri/dwSpark/parquet/DW_DRI.DWF_PRESTACAO_CONTA.parquet')

### Projeto

In [65]:
##Prefixo
pref_dwd_processo_projeto_proj = 'proj_'
pref_dwd_unidade_organizacional_proj = 'uo_'
pref_dwd_programa_proj = 'programa_'
pref_dwd_edital = 'edital_'
pref_dwd_tipo_beneficio = 'tp_beneficio_'
pref_ies_principal = 'ies_princip_'
pref_geografia_resp_principal = 'geo_resp_princp_'
pref_responsavel_principal = 'resp_princp_'
pref_dwd_situacao_processo_proj = 'st_proc_'

In [66]:
dwd_processo_projeto_proj = dwd_processo_projeto.select([f.col(c) for c in dwd_processo_projeto.columns if 'SK' in c] + \
    [f.col(c).alias(c.lower()) for c in dwd_processo_projeto.columns if 'SK' not in c])
dwd_unidade_organizacional_proj = dwd_unidade_organizacional.select(['SK_UNIDADE_ORGANIZACIONAL']+[f.col(c).alias(pref_dwd_unidade_organizacional_proj+c.lower()) for c in dwd_unidade_organizacional.columns])
dwd_programa_proj = dwd_programa.select(['SK_PROGRAMA']+[f.col(c).alias(pref_dwd_programa_proj+c.lower()) for c in dwd_programa.columns])
dwd_edital_proj = dwd_edital.select(['SK_EDITAL']+[f.col(c).alias(pref_dwd_edital+c.lower()) for c in dwd_edital.columns])
dwd_tipo_beneficio_proj = dwd_tipo_beneficio.select(['SK_TIPO_BENEFICIO']+[f.col(c).alias(pref_dwd_tipo_beneficio+c.lower()) for c in dwd_tipo_beneficio.columns])
ies_principal = dwd_instituicao_ensino.withColumnRenamed('SK_INSTITUICAO_ENSINO', 'SK_IES_PRINCIPAL')
ies_principal = ies_principal.select(['SK_IES_PRINCIPAL']+[f.col(c).alias(pref_ies_principal+c.lower()) for c in ies_principal.columns])
geografia_resp_principal = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEOGRAFIA_RESP_PRINCIPAL')
geografia_resp_principal = geografia_resp_principal.select(['SK_GEOGRAFIA_RESP_PRINCIPAL']+[f.col(c).alias(pref_geografia_resp_principal+c.lower()) for c in geografia_resp_principal.columns])
responsavel_principal = dwd_pessoa.withColumnRenamed('SK_PESSOA', 'SK_RESPONSAVEL_PRINCIPAL')
responsavel_principal = responsavel_principal.select(['SK_RESPONSAVEL_PRINCIPAL']+[f.col(c).alias(pref_responsavel_principal+c.lower()) for c in responsavel_principal.columns])
dwd_situacao_processo_proj = dwd_situacao_processo.select(['SK_SITUACAO_PROCESSO']+[f.col(c).alias(pref_dwd_situacao_processo_proj+c.lower()) for c in dwd_situacao_processo.columns])

In [67]:
projeto = \
dwd_processo_projeto_proj \
.join(dwd_unidade_organizacional_proj, 'SK_UNIDADE_ORGANIZACIONAL') \
.join(dwd_programa_proj, 'SK_PROGRAMA') \
.join(dwd_edital_proj, 'SK_EDITAL') \
.join(dwd_tipo_beneficio_proj, 'SK_TIPO_BENEFICIO') \
.join(ies_principal, 'SK_IES_PRINCIPAL') \
.join(geografia_resp_principal, 'SK_GEOGRAFIA_RESP_PRINCIPAL') \
.join(responsavel_principal, 'SK_RESPONSAVEL_PRINCIPAL') \
.join(dwd_situacao_processo_proj, 'SK_SITUACAO_PROCESSO')
## Remove as SKs
projeto = \
projeto \
.select([f.col(c) for c in projeto.columns if 'sk' not in c.lower()])
## Adiciona o prefixo 'proj_' as colunas
projeto = \
projeto \
.select([f.col(c).alias(pref_dwd_processo_projeto_proj+c) for c in projeto.columns])
## Transforma as colunas 'dt' em data
proj_columns = []
for i in [f.col(c) for c in projeto.columns]:
    if '_dt_' in str(i).lower():
        proj_columns.append(i.cast('date'))
    else:
        proj_columns.append(i)
projeto = \
projeto \
.select(proj_columns)

### Bolsas

In [68]:
##Prefixo
pref_dwd_processo_bolsa_bolsa = 'bolsa_'
pref_dwd_unidade_organizacional_bolsa = 'uo_'
pref_dwd_programa_bolsa = 'programa_'
pref_dwd_edital_bolsa = 'edital_'
pref_dwd_tipo_beneficio_bolsa = 'tp_beneficio_'
pref_bolsista_bolsa = 'beneficiario_'
pref_geografia_exterior_bolsa = 'geo_exterior_'
pref_geografia_brasil_bolsa = 'geo_brasil_'
pref_ies_origem_bolsa = 'ies_ori_'
pref_geografia_ies_origem_bolsa = 'geo_ies_ori_'
pref_ies_estudo_bolsa = 'ies_estu_'
pref_geografia_ies_estudo_bolsa = 'geo_ies_estu_'
pref_dwd_modalidade_bolsa = 'modalidade_'
pref_dwd_situacao_processo_bolsa = 'st_proc_'

In [69]:
dwd_processo_bolsa_bolsa = dwd_processo_bolsa.select([f.col(c) for c in dwd_processo_bolsa.columns if 'SK' in c] + \
    [f.col(c).alias(c.lower()) for c in dwd_processo_bolsa.columns if 'SK' not in c])
dwd_unidade_organizacional_bolsa = dwd_unidade_organizacional.select(['SK_UNIDADE_ORGANIZACIONAL']+[f.col(c).alias(pref_dwd_unidade_organizacional_bolsa+c.lower()) for c in dwd_unidade_organizacional.columns])
dwd_programa_bolsa = dwd_programa.select(['SK_PROGRAMA']+[f.col(c).alias(pref_dwd_programa_bolsa+c.lower()) for c in dwd_programa.columns])
dwd_edital_bolsa = dwd_edital.select(['SK_EDITAL']+[f.col(c).alias(pref_dwd_edital_bolsa+c.lower()) for c in dwd_edital.columns])
dwd_tipo_beneficio_bolsa = dwd_tipo_beneficio.select(['SK_TIPO_BENEFICIO']+[f.col(c).alias(pref_dwd_tipo_beneficio_bolsa+c.lower()) for c in dwd_tipo_beneficio.columns])
bolsista = dwd_pessoa.withColumnRenamed('SK_PESSOA', 'SK_BOLSISTA')
bolsista = bolsista.select(['SK_BOLSISTA']+[f.col(c).alias(pref_bolsista_bolsa+c.lower()) for c in bolsista.columns])
geografia_exterior = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEO_EXTERIOR')
geografia_exterior = geografia_exterior.select(['SK_GEO_EXTERIOR']+[f.col(c).alias(pref_geografia_exterior_bolsa+c.lower()) for c in geografia_exterior.columns])
geografia_brasil = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEO_BRASIL')
geografia_brasil = geografia_brasil.select(['SK_GEO_BRASIL']+[f.col(c).alias(pref_geografia_brasil_bolsa+c.lower()) for c in geografia_brasil.columns])
ies_origem = dwd_instituicao_ensino.withColumnRenamed('SK_INSTITUICAO_ENSINO', 'SK_IES_ORIGEM')
ies_origem = ies_origem.select(['SK_IES_ORIGEM']+[f.col(c).alias(pref_ies_origem_bolsa+c.lower()) for c in ies_origem.columns])
geografia_ies_origem = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEOGRAFIA_IES_ORIGEM')
geografia_ies_origem = geografia_ies_origem.select(['SK_GEOGRAFIA_IES_ORIGEM']+[f.col(c).alias(pref_geografia_ies_origem_bolsa+c.lower()) for c in geografia_ies_origem.columns])
ies_estudo = dwd_instituicao_ensino.withColumnRenamed('SK_INSTITUICAO_ENSINO', 'SK_IES_ESTUDO')
ies_estudo = ies_estudo.select(['SK_IES_ESTUDO']+[f.col(c).alias(pref_ies_estudo_bolsa+c.lower()) for c in ies_estudo.columns])
geografia_ies_estudo = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEOGRAFIA_IES_ESTUDO')
geografia_ies_estudo = geografia_ies_estudo.select(['SK_GEOGRAFIA_IES_ESTUDO']+[f.col(c).alias(pref_geografia_ies_estudo_bolsa+c.lower()) for c in geografia_ies_estudo.columns])
dwd_modalidade_bolsa_bolsa = dwd_modalidade_bolsa.select(['SK_MODALIDADE_BOLSA']+[f.col(c).alias(pref_dwd_modalidade_bolsa+c.lower()) for c in dwd_modalidade_bolsa.columns])
dwd_situacao_processo_bolsa = dwd_situacao_processo.select(['SK_SITUACAO_PROCESSO']+[f.col(c).alias(pref_dwd_situacao_processo_bolsa+c.lower()) for c in dwd_situacao_processo.columns])

In [70]:
bolsa = \
dwd_processo_bolsa_bolsa \
.join(dwd_unidade_organizacional_bolsa, 'SK_UNIDADE_ORGANIZACIONAL') \
.join(dwd_programa_bolsa, 'SK_PROGRAMA') \
.join(dwd_edital_bolsa, 'SK_EDITAL') \
.join(dwd_tipo_beneficio_bolsa, 'SK_TIPO_BENEFICIO') \
.join(bolsista, 'SK_BOLSISTA') \
.join(geografia_exterior, 'SK_GEO_EXTERIOR') \
.join(geografia_brasil, 'SK_GEO_BRASIL') \
.join(ies_origem, 'SK_IES_ORIGEM') \
.join(geografia_ies_origem, 'SK_GEOGRAFIA_IES_ORIGEM') \
.join(ies_estudo, 'SK_IES_ESTUDO') \
.join(dwd_modalidade_bolsa_bolsa, 'SK_MODALIDADE_BOLSA') \
.join(dwd_situacao_processo_bolsa, 'SK_SITUACAO_PROCESSO')
#.join(geografia_ies_estudo, 'SK_GEOGRAFIA_IES_ESTUDO') \
## Remove as SKs
bolsa = \
bolsa \
.select([f.col(c) for c in bolsa.columns if 'sk' not in c.lower()])
## Adiciona o prefixo 'bolsa_' as colunas
bolsa = \
bolsa \
.select([f.col(c).alias(pref_dwd_processo_bolsa_bolsa+c) for c in bolsa.columns])
## Transforma as colunas 'dt' em data
bolsa_columns = []
for i in [f.col(c) for c in bolsa.columns]:
    if '_dt_' in str(i).lower():
        bolsa_columns.append(i.cast('date'))
    else:
        bolsa_columns.append(i)
bolsa = \
bolsa \
.select(bolsa_columns)

### AUXPE

In [71]:
##Prefixo
pref_dwd_processo_auxpe_auxpe = 'auxpe_'
pref_dwd_unidade_organizacional_auxpe = 'uo_'
pref_dwd_programa_auxpe = 'programa_'
pref_dwd_edital_auxpe = 'edital_'
pref_dwd_tipo_beneficio_auxpe = 'tp_beneficio_'
pref_beneficiario_auxpe = 'beneficiario_'
pref_geografia_exterior_auxpe = 'geo_exterior_'
pref_geografia_brasil_auxpe = 'geo_brasil_'
pref_ies_origem_auxpe = 'ies_ori_'
pref_geografia_ies_origem_auxpe = 'geo_ies_ori_'
pref_ies_estudo_auxpe = 'ies_estu_'
pref_geografia_ies_estudo_auxpe = 'geo_ies_estu_'
pref_dwd_situacao_processo_auxpe = 'st_proc_'

In [72]:
dwd_processo_auxpe_auxpe = dwd_processo_auxpe.select([f.col(c) for c in dwd_processo_auxpe.columns if 'SK' in c] + \
    [f.col(c).alias(c.lower()) for c in dwd_processo_auxpe.columns if 'SK' not in c])
dwd_unidade_organizacional_auxpe = dwd_unidade_organizacional.select(['SK_UNIDADE_ORGANIZACIONAL']+[f.col(c).alias(pref_dwd_unidade_organizacional_auxpe+c.lower()) for c in dwd_unidade_organizacional.columns])
dwd_programa_auxpe = dwd_programa.select(['SK_PROGRAMA']+[f.col(c).alias(pref_dwd_programa_auxpe+c.lower()) for c in dwd_programa.columns])
dwd_edital_auxpe = dwd_edital.select(['SK_EDITAL']+[f.col(c).alias(pref_dwd_edital_auxpe+c.lower()) for c in dwd_edital.columns])
dwd_tipo_beneficio_auxpe = dwd_tipo_beneficio.select(['SK_TIPO_BENEFICIO']+[f.col(c).alias(pref_dwd_tipo_beneficio_auxpe+c.lower()) for c in dwd_tipo_beneficio.columns])
beneficiario = dwd_pessoa.withColumnRenamed('SK_PESSOA', 'SK_BENEFICIARIO')
beneficiario = beneficiario.select(['SK_BENEFICIARIO']+[f.col(c).alias(pref_beneficiario_auxpe+c.lower()) for c in beneficiario.columns])
geografia_exterior = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEO_EXTERIOR')
geografia_exterior = geografia_exterior.select(['SK_GEO_EXTERIOR']+[f.col(c).alias(pref_geografia_exterior_auxpe+c.lower()) for c in geografia_exterior.columns])
geografia_brasil = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEO_BRASIL')
geografia_brasil = geografia_brasil.select(['SK_GEO_BRASIL']+[f.col(c).alias(pref_geografia_brasil_auxpe+c.lower()) for c in geografia_brasil.columns])
ies_origem = dwd_instituicao_ensino.withColumnRenamed('SK_INSTITUICAO_ENSINO', 'SK_IES_ORIGEM')
ies_origem = ies_origem.select(['SK_IES_ORIGEM']+[f.col(c).alias(pref_ies_origem_auxpe+c.lower()) for c in ies_origem.columns])
geografia_ies_origem = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEOGRAFIA_IES_ORIGEM')
geografia_ies_origem = geografia_ies_origem.select(['SK_GEOGRAFIA_IES_ORIGEM']+[f.col(c).alias(pref_geografia_ies_origem_auxpe+c.lower()) for c in geografia_ies_origem.columns])
ies_estudo = dwd_instituicao_ensino.withColumnRenamed('SK_INSTITUICAO_ENSINO', 'SK_IES_ESTUDO')
ies_estudo = ies_estudo.select(['SK_IES_ESTUDO']+[f.col(c).alias(pref_ies_estudo_auxpe+c.lower()) for c in ies_estudo.columns])
geografia_ies_estudo = dwd_geografia.withColumnRenamed('SK_GEOGRAFIA', 'SK_GEOGRAFIA_IES_ESTUDO')
geografia_ies_estudo = geografia_ies_estudo.select(['SK_GEOGRAFIA_IES_ESTUDO']+[f.col(c).alias(pref_geografia_ies_estudo_auxpe+c.lower()) for c in geografia_ies_estudo.columns])
dwd_situacao_processo_auxpe = dwd_situacao_processo.select(['SK_SITUACAO_PROCESSO']+[f.col(c).alias(pref_dwd_situacao_processo_auxpe+c.lower()) for c in dwd_situacao_processo.columns])

In [73]:
auxpe = \
dwd_processo_auxpe_auxpe \
.join(dwd_unidade_organizacional_auxpe, 'SK_UNIDADE_ORGANIZACIONAL') \
.join(dwd_programa_auxpe, 'SK_PROGRAMA') \
.join(dwd_edital_auxpe, 'SK_EDITAL') \
.join(dwd_tipo_beneficio_auxpe, 'SK_TIPO_BENEFICIO') \
.join(beneficiario, 'SK_BENEFICIARIO') \
.join(geografia_exterior, 'SK_GEO_EXTERIOR') \
.join(geografia_brasil, 'SK_GEO_BRASIL') \
.join(ies_origem, 'SK_IES_ORIGEM') \
.join(geografia_ies_origem, 'SK_GEOGRAFIA_IES_ORIGEM') \
.join(ies_estudo, 'SK_IES_ESTUDO') \
.join(dwd_situacao_processo_auxpe, 'SK_SITUACAO_PROCESSO')
## Remove as SKs
auxpe = \
auxpe \
.select([f.col(c) for c in auxpe.columns if 'sk' not in c.lower()])
## Adiciona o prefixo 'auxpe_' as colunas
auxpe = \
auxpe \
.select([f.col(c).alias(pref_dwd_processo_auxpe_auxpe+c) for c in auxpe.columns])
## Transforma as colunas 'dt' em data
auxpe_columns = []
for i in [f.col(c) for c in auxpe.columns]:
    if '_dt_' in str(i).lower():
        auxpe_columns.append(i.cast('date'))
    else:
        auxpe_columns.append(i)
auxpe = \
auxpe \
.select(auxpe_columns)

### Union Bolsa e AUXPE

In [86]:
auxpe_union = auxpe.select([f.col(c).alias(str(c).replace("auxpe_", "")) for c in auxpe.columns]).withColumnRenamed('id_processo_auxpe', 'id_processo')
bolsa_union = bolsa.select([f.col(c).alias(str(c).replace("bolsa_", "")) for c in bolsa.columns]).withColumnRenamed('id_processo_bolsa', 'id_processo')
## Renomeia as colunas exclusivas AUXPE
colunas_processo = []
colunas_auxpe = []
for i in auxpe_union.columns:
    if i not in bolsa_union.columns:
        colunas_auxpe.append(f.col(i).alias('auxpe_'+i))
    if i in bolsa_union.columns:
        colunas_processo.append(f.col(i).alias('processo_'+i))
colunas_processo_auxpe = colunas_processo+colunas_auxpe
## Renomeia as colunas exclusivas Bolsa
colunas_processo = []
colunas_bolsa = []
for i in bolsa_union.columns:
    if i not in auxpe_union.columns:
        colunas_bolsa.append(f.col(i).alias('bolsa_'+i))
    if i in auxpe_union.columns:
        colunas_processo.append(f.col(i).alias('processo_'+i))
colunas_processo_bolsa = colunas_processo+colunas_bolsa
auxpe_union = auxpe_union.select(colunas_processo_auxpe)
bolsa_union = bolsa_union.select(colunas_processo_bolsa)

In [89]:
bolsa_union.unionByName(auxpe_union, allowMissingColumns=True).show()

+--------------------+--------------------+--------------------+-----------------------+--------------------------------+------------------+-----------------------+-----------------------+----------------------------+-----------------+-------------------------+--------------------------+-------------------------------------+------------------------+------------------------+--------------------------------+--------------------------------+--------------------------+--------------------------+--------------------+--------------------+-------------------------------+--------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-------------------------------------+--------------------------+----------------------------------+-----------------------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------------

In [92]:
%%sh
sudo apt install git

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


CalledProcessError: Command 'b'sudo apt install git\n'' returned non-zero exit status 1.

### Pagamento

In [16]:
##Prefixo
pref_dwf_pagamento = 'pgt_'
pref_dwd_rubrica_pgt = 'rubrica_'
pref_dwd_tipo_solicitacao_pgt = 'tp_solicitacao_'
pref_dwd_moeda_pgt = 'moeda_'

In [17]:
dwf_pagamento_pagamento = dwf_pagamento.select([f.col(c) for c in dwf_pagamento.columns if 'SK' in c] + \
    [f.col(c).alias(c.lower()) for c in dwf_pagamento.columns if 'SK' not in c])
dwd_rubrica_pgt = dwd_rubrica.select(['SK_RUBRICA']+[f.col(c).alias(pref_dwd_rubrica_pgt+c.lower()) for c in dwd_rubrica.columns])
dwd_tipo_solicitacao_pgt = dwd_tipo_solicitacao.select(['SK_TIPO_SOLICITACAO']+[f.col(c).alias(pref_dwd_tipo_solicitacao_pgt+c.lower()) for c in dwd_tipo_solicitacao.columns])
dwd_moeda_pgt = dwd_moeda.select(['SK_MOEDA']+[f.col(c).alias(pref_dwd_moeda_pgt+c.lower()) for c in dwd_moeda.columns])

In [18]:
pagamento = \
dwf_pagamento_pagamento \
.join(dwd_rubrica_pgt, 'SK_RUBRICA') \
.join(dwd_tipo_solicitacao_pgt, 'SK_TIPO_SOLICITACAO') \
.join(dwd_moeda_pgt, 'SK_MOEDA') \
.withColumn('dt_pagamento', f.to_date(f.col('SK_DATA_PAGAMENTO').cast('string'), 'yyyyMMdd')) \
.withColumn('dt_referencia', f.to_date(f.col('SK_DATA_REFERENCIA').cast('string'), 'yyyyMMdd'))
## Remove as SKs
pagamento = \
pagamento \
.select([f.col(c) for c in pagamento.columns if 'sk' not in c.lower()])
## Adiciona o prefixo 'pagamento_' as colunas
pagamento = \
pagamento \
.select([f.col(c).alias(pref_dwf_pagamento+c) for c in pagamento.columns])
## Transforma as colunas 'dt' em data
pagamento_columns = []
for i in [f.col(c) for c in pagamento.columns]:
    if '_dt_' in str(i).lower():
        pagamento_columns.append(i.cast('date'))
    else:
        pagamento_columns.append(i)
pagamento = \
pagamento \
.select(pagamento_columns)

### Salva df.csv.zip

In [4]:
##Salva CSV
def salva_csv(df, nm):
    compression_opts = dict(method='zip', archive_name=f'{nm}.csv')
    df.toPandas().to_csv(path_or_buf=f'./{nm}.zip', sep=';', na_rep='', float_format=None, columns=None, header=True, index=False, index_label=None, mode='w', encoding=None, compression=compression_opts, quoting=None, quotechar='"', line_terminator=None, chunksize=200000, date_format=None, doublequote=True, escapechar=None, decimal=',', errors='strict', storage_options=None)

In [19]:
%%time
salva_csv(projeto, 'projeto')

CPU times: user 932 ms, sys: 0 ns, total: 932 ms
Wall time: 7.45 s


In [20]:
%%time
salva_csv(bolsa, 'bolsa')

CPU times: user 30.6 s, sys: 415 ms, total: 31 s
Wall time: 44.3 s


In [21]:
%%time
salva_csv(auxpe, 'auxpe')

CPU times: user 1.43 s, sys: 11.9 ms, total: 1.44 s
Wall time: 6.47 s


In [22]:
%%time
salva_csv(pagamento, 'pagamento')

CPU times: user 58.9 s, sys: 916 ms, total: 59.8 s
Wall time: 1min 10s


In [23]:
ll

total 249148
-rw-rw-r-- 1 bosco    868424 May 16 05:37 auxpe.zip
-rw-rw-r-- 1 bosco  21590069 May 16 05:36 bolsa.zip
-rw-rw-r-- 1 bosco   7437474 May 16 05:38 pagamento.zip
-rw-rw-r-- 1 bosco    816722 May 16 05:36 projeto.zip
-rw-rw-r-- 1 bosco 224374704 Feb 22 02:45 spark-3.1.1-bin-hadoop2.7.tgz
-rw-rw-r-- 1 bosco     25368 May 16 05:38 Untitled.ipynb
